In [21]:
data <- read.csv("../data/support2.csv")
head(data)

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,⋯,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
,<dbl>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<dbl>
1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,11,⋯,1.1999512,141,7.459961,NA,NA,NA,7,7,,7
2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,5.5000000,132,7.250000,NA,NA,NA,NA,1,<2 mo. follow-up,1
3,52.74698,1,female,0,17,47,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,2.0000000,134,7.459961,NA,NA,NA,1,0,<2 mo. follow-up,0
4,42.38498,1,female,0,3,133,Lung Cancer,Cancer,2,11,⋯,0.7999268,139,NA,NA,NA,NA,0,0,no(M2 and SIP pres),0
5,79.88495,0,female,0,16,2029,ARF/MOSF w/Sepsis,ARF/MOSF,1,NA,⋯,0.7999268,143,7.509766,NA,NA,NA,NA,2,no(M2 and SIP pres),2
6,93.01599,1,male,1,4,4,Coma,Coma,1,14,⋯,0.6999512,140,7.659180,NA,NA,NA,NA,1,<2 mo. follow-up,1


In [22]:
data$pafi[is.na(data$pafi)] <- 333.3
data$alb[is.na(data$alb)] <- 3.5
data$bun[is.na(data$bun)] <- 6.51
data$urine[is.na(data$urine)] <- 2502

In [23]:
data[] <- lapply(data, function(x) {
  if (is.numeric(x)) {
    x[is.na(x)] <- median(x, na.rm = TRUE) 
  }
  return(x)
})
head(data)

,age,death,sex,hospdead,slos,d.time,dzgroup,dzclass,num.co,edu,⋯,crea,sod,ph,glucose,bun,urine,adlp,adls,sfdm2,adlsc
,<dbl>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,62.84998,0,male,0,5,2029,Lung Cancer,Cancer,0,11,⋯,1.1999512,141,7.459961,135,6.51,2502,7,7,,7
2,60.33899,1,female,1,4,4,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,5.5000000,132,7.250000,135,6.51,2502,0,1,<2 mo. follow-up,1
3,52.74698,1,female,0,17,47,Cirrhosis,COPD/CHF/Cirrhosis,2,12,⋯,2.0000000,134,7.459961,135,6.51,2502,1,0,<2 mo. follow-up,0
4,42.38498,1,female,0,3,133,Lung Cancer,Cancer,2,11,⋯,0.7999268,139,7.419922,135,6.51,2502,0,0,no(M2 and SIP pres),0
5,79.88495,0,female,0,16,2029,ARF/MOSF w/Sepsis,ARF/MOSF,1,12,⋯,0.7999268,143,7.509766,135,6.51,2502,0,2,no(M2 and SIP pres),2
6,93.01599,1,male,1,4,4,Coma,Coma,1,14,⋯,0.6999512,140,7.659180,135,6.51,2502,0,1,<2 mo. follow-up,1


In [24]:
colonnes_chaine <- sapply(data, is.character)

nombres_vides <- sapply(data[, colonnes_chaine], function(x) sum(x == ""))

# Afficher les résultats
resultats <- data.frame(
  colonne = names(nombres_vides),
  nombre_vides = nombres_vides
)
resultats

,colonne,nombre_vides
,<chr>,<int>
sex,sex,0
dzgroup,dzgroup,0
dzclass,dzclass,0
income,income,2982
race,race,42
ca,ca,0
dnr,dnr,30
sfdm2,sfdm2,1400


In [25]:
data <- data[, !names(data) %in% "income"]
#data <- data[, !names(data) %in% "sfdm2"]
data <- data[!(is.na(data$sfdm2) | data$sfdm2 == ""), ]
data <- data[!(is.na(data$race) | data$race == ""), ]
data <- data[!(is.na(data$dnr) | data$dnr == ""), ]

colonnes_chaine <- sapply(data, is.character)

nombres_vides <- sapply(data[, colonnes_chaine], function(x) sum(x == ""))

# Afficher les résultats
resultats <- data.frame(
  colonne = names(nombres_vides),
  nombre_vides = nombres_vides
)
resultats

,colonne,nombre_vides
,<chr>,<int>
sex,sex,0
dzgroup,dzgroup,0
dzclass,dzclass,0
race,race,0
ca,ca,0
dnr,dnr,0
sfdm2,sfdm2,0


In [26]:
A <- ifelse(data$hospdead > 0.5, 1, 0)
A <- A[A == 1]
B <- ifelse(data$hospdead < 0.5, 1, 0)
B <- B[B == 1]

A_t = length(A)/nrow(data)
B_t = length(B)/nrow(data)

inv_At = 1/A_t *B_t
inv_Bt = 1

inv_At

[1] 2.333043

In [27]:
library(caret)

set.seed(123)
index <- createDataPartition(data$hospdead, p = 0.7, list = FALSE)

train_data2 <- data[index, ]
test_data2 <- data[-index, ]

In [28]:
weights <- ifelse(train_data2$hospdead == 1, 1, inv_At)

In [29]:
library(glmnet)


In [30]:
formula <- hospdead ~ . - 1

x_train <- model.matrix(formula, data = train_data2)
y_train <- train_data2$hospdead

x_test <- model.matrix(formula, data = test_data2)
y_test <- test_data2$hospdead

In [39]:
ridge_model <- glmnet(x_train, y_train, family = "binomial", alpha = 0)

cv_ridge <- cv.glmnet(x_train, y_train, family = "binomial", alpha = 0)

best_lambda_ridge <- cv_ridge$lambda.min

final_ridge_model <- glmnet(x_train, y_train, family = "binomial", alpha = 0, lambda = best_lambda_ridge, weights = weights)

In [70]:
#cv_lasso <- cv.glmnet(x_train, y_train, family = "binomial", alpha = 1)

#best_lambda_lasso <- cv_lasso$lambda.min

final_lasso_model <- glmnet(x_train, y_train, family = "binomial", alpha = 1)

Warning message:
“from glmnet C++ code (error code -88); Convergence for 88th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned”


In [68]:
final_lasso_model


Call:  glmnet(x = x_train, y = y_train, family = "binomial", weights = weights,      alpha = 1, lambda = best_lambda_lasso) 

   Df  %Dev   Lambda
1   0  0.00 0.181300
2   3  6.76 0.165200
3   3 13.37 0.150500
4   3 18.68 0.137200
5   3 23.05 0.125000
6   3 26.73 0.113900
7   4 29.90 0.103800
8   4 33.19 0.094550
9   5 36.14 0.086150
10  5 38.84 0.078490
11  5 41.26 0.071520
12  5 43.43 0.065170
13  5 45.40 0.059380
14  5 47.19 0.054100
15  6 49.03 0.049300
16  7 50.94 0.044920
17  7 52.65 0.040930
18  7 54.18 0.037290
19  7 55.57 0.033980
20  7 56.81 0.030960
21  8 57.96 0.028210
22  9 59.19 0.025700
23  9 60.30 0.023420
24  9 61.30 0.021340
25  9 62.18 0.019440
26 10 63.05 0.017720
27 11 63.82 0.016140
28 11 64.51 0.014710
29 12 65.13 0.013400
30 12 65.68 0.012210
31 12 66.18 0.011130
32 12 66.63 0.010140
33 14 67.03 0.009237
34 16 67.42 0.008417
35 17 67.79 0.007669
36 19 68.13 0.006988
37 22 68.48 0.006367
38 22 68.81 0.005801
39 22 69.12 0.005286
40 23 69.42 0.004816
41 23 69.69 

In [72]:
best_lambda_lasso <- final_lasso_model$lambda.min
arg_min <- which.min(final_lasso_model$lambda)

final_lasso_model <- glmnet(x_train, y_train, family = "binomial", alpha = 1, lambda = best_lambda_lasso, weights = weights)

Warning message:
“from glmnet C++ code (error code -82); Convergence for 82th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned”


In [79]:
arg_min

[1] 87

In [40]:
ridge_preds <- predict(final_ridge_model, s = best_lambda_ridge, newx = x_test, type = "response")
ridge_classes <- ifelse(ridge_preds > 0.5, 1, 0)

conf_matrix_ridge <- table(Actual = y_test, Predicted = ridge_classes)

In [41]:
print("Confusion Matrix for Ridge:")
print(conf_matrix_ridge)

accuracy_ridge <- sum(diag(conf_matrix_ridge)) / sum(conf_matrix_ridge)

print(paste("Accuracy for Ridge:", accuracy_ridge))

[1] "Confusion Matrix for Ridge:"
      Predicted
Actual    0    1
     0 1576   38
     1  188  491
[1] "Accuracy for Ridge: 0.901439162668993"


In [77]:
lasso_preds <- predict(final_lasso_model[arg_min], s = best_lambda_lasso, newx = x_test, type = "response")
lasso_classes <- ifelse(lasso_preds > 0.5, 1, 0)

conf_matrix_lasso <- table(Actual = y_test, Predicted = lasso_classes)

ERROR: Error in UseMethod("predict"): pas de méthode pour 'predict' applicable pour un objet de classe "NULL"


In [75]:
length(y_test)

[1] 2293

In [76]:
length(lasso_classes)

[1] 185733

In [67]:
lasso_preds

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,⋯,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80
5,0.1562513,0.1516469,0.1422986,0.13319000,0.12446573,0.11619354,0.10727731,0.08825831,0.07193419,0.05804534,⋯,9.549713e-124,1.419638e-134,8.168105e-147,9.093178e-160,9.288869e-174,6.744672e-189,3.034610e-205,1.223796e-222,3.547452e-241,9.490362e-261
6,0.1562513,0.1597167,0.1471345,0.13541595,0.12457252,0.11457480,0.10575651,0.10147396,0.09808927,0.09606796,⋯,1.957542e-01,2.047436e-01,2.149876e-01,2.255423e-01,2.365106e-01,2.484357e-01,2.614184e-01,2.751790e-01,2.902739e-01,3.060290e-01
12,0.1562513,0.1550046,0.1560927,0.15579073,0.15449947,0.15248097,0.15022835,0.15080215,0.15154883,0.15305332,⋯,3.966627e-01,4.104588e-01,4.247428e-01,4.392848e-01,4.536090e-01,4.681350e-01,4.829934e-01,4.974782e-01,5.113902e-01,5.245708e-01
13,0.1562513,0.1376618,0.1186057,0.10275401,0.08939490,0.07803077,0.06861491,0.06351545,0.05848599,0.05312680,⋯,2.181335e-10,3.483275e-11,4.368368e-12,4.791735e-13,4.386179e-14,3.271773e-15,1.962449e-16,9.750187e-18,3.919788e-19,1.302022e-20
15,0.1562513,0.1638022,0.1701025,0.17420551,0.17674000,0.17813338,0.17694345,0.15726322,0.14000641,0.12657286,⋯,1.399053e-115,8.104962e-126,2.161391e-137,1.242671e-149,7.411358e-163,3.633205e-177,1.291859e-192,4.713358e-209,1.429433e-226,4.581509e-245
18,0.1562513,0.1979581,0.2454830,0.29118188,0.33469302,0.37587751,0.41474887,0.45172668,0.48615822,0.51779996,⋯,9.898658e-01,9.903218e-01,9.907797e-01,9.911947e-01,9.916244e-01,9.920352e-01,9.924275e-01,9.928076e-01,9.932046e-01,9.935827e-01
19,0.1562513,0.1559960,0.1586003,0.15963184,0.15952259,0.15855780,0.15725656,0.15879162,0.16043795,0.16279594,⋯,4.771358e-01,4.916760e-01,5.071983e-01,5.233326e-01,5.392083e-01,5.555159e-01,5.723862e-01,5.891295e-01,6.040462e-01,6.178253e-01
21,0.1562513,0.1369620,0.1176889,0.10171402,0.08829319,0.07690858,0.06746440,0.06199887,0.05668739,0.05115853,⋯,8.105366e-15,5.694890e-16,2.831644e-17,1.159729e-18,3.668751e-20,8.660394e-22,1.497814e-23,1.987315e-25,1.978841e-27,1.513800e-29
22,0.1562513,0.1645466,0.1714622,0.17632018,0.17963769,0.18176301,0.18316543,0.18617882,0.18913192,0.19260723,⋯,2.978690e-01,2.898263e-01,2.791174e-01,2.691188e-01,2.581122e-01,2.465403e-01,2.343562e-01,2.206535e-01,2.053502e-01,1.891824e-01
26,0.1562513,0.1462462,0.1431229,0.13928561,0.13499469,0.13041528,0.12505083,0.11338246,0.10155910,0.08925503,⋯,1.102319e-65,2.275018e-71,8.616399e-78,1.347346e-84,6.088247e-92,6.829089e-100,1.756474e-108,1.272904e-117,2.318657e-127,1.206868e-137


In [59]:
print("Confusion Matrix for Lasso:")
print(conf_matrix_lasso)

accuracy_lasso <- sum(diag(conf_matrix_lasso)) / sum(conf_matrix_lasso)

print(paste("Accuracy for Lasso:", accuracy_lasso))

[1] "Confusion Matrix for Lasso:"


      Predicted
Actual    0    1
     0 1530   84
     1   69  610
[1] "Accuracy for Lasso: 0.933275185346707"
